In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 9.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf
import glob
import os

In [3]:
path = '/content/drive/MyDrive/Master Project 295A/final text dataset/processed_files'
all_files = glob.glob(os.path.join(path, "*.csv")) 
df_from_each_file = (pd.read_csv(f)[:200] for f in all_files)
concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)
concatenated_df = concatenated_df.drop(["Unnamed: 0"], axis=1)
concatenated_df.columns = ['category', 'text']

In [4]:
concatenated_df

,category,text
0,book,﻿How can we free ourselves from mental sufferi...
1,book,"﻿ The New Girl Webster Grove, Book 1 Tracie ..."
2,book,﻿DEATH BY CHOCOLATE Copyright ©2011 Sally Bern...
3,book,﻿ Always Know What to Say Easy Ways to Appro...
4,book,﻿ A Little Bit of Déjà Vu The Return to Redem...
...,...,...
618,legal,Mutual Non-Disclosure Agreement\nThis AGREEMEN...
619,legal,Non-Disclosure Agreement\n Moscow ___.___.2016...
620,legal,[European Space Agency]\nand\n[Contractor]\nNo...
621,legal,NON DISCLOSURE AGREEMENT\nAGREEMENT between Am...


In [5]:
!pip install clean-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171048 sha256=20dd03ccd6812147a315848302ca233b7588dc1cfb285a6c7ff3adeb937cb186
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji


In [6]:
from cleantext.sklearn import CleanTransformer
cleaner = CleanTransformer(no_punct=True, lower=True,fix_unicode=True,no_line_breaks=True)
concatenated_df['cleaned_text'] = cleaner.transform(concatenated_df['text'])

In [7]:
data_cleaned = concatenated_df.drop(['text'],axis = 1)

In [8]:
data_cleaned["category"] = data_cleaned["category"].astype('category')
data_cleaned["category"] = data_cleaned["category"].cat.codes

In [9]:
data_cleaned

,category,cleaned_text
0,0,how can we free ourselves from mental sufferin...
1,0,the new girl webster grove book 1 tracie pucke...
2,0,death by chocolate copyright ©️2011 sally bern...
3,0,always know what to say easy ways to approach ...
4,0,a little bit of deja vu the return to redempti...
...,...,...
618,1,mutual nondisclosure agreement this agreement ...
619,1,nondisclosure agreement moscow 2016 the autono...
620,1,european space agency and contractor nondisclo...
621,1,non disclosure agreement agreement between ame...


In [10]:
data_cleaned["category"].unique()

array([0, 4, 2, 3, 1], dtype=int8)

In [14]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import TFBertForSequenceClassification, BertTokenizer

# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=5)

# Load the cleaned data
X = data_cleaned['cleaned_text'].values
y = data_cleaned['category'].values

# Convert the text data to tokens
max_len = 200
X_tokenized = tokenizer(X.tolist(), padding=True, truncation=True, max_length=max_len)




All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_tokenized['input_ids'], y, test_size=0.2, random_state=42)

# Create TensorFlow datasets for the training and validation sets
batch_size = 16
train_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': X_train}, y_train)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': X_val}, y_val)).batch(batch_size)

# Compile the model
bert_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
                   loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                   metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Fine-tune the model on the training set
history = bert_model.fit(train_dataset, epochs=1, validation_data=val_dataset)

# Evaluate the model on the validation set
loss, accuracy = bert_model.evaluate(val_dataset)
print(f"Validation loss: {loss:.4f}, Validation accuracy: {accuracy:.4f}")


8/8 [==============================] - 87s 11s/step - loss: 0.5167 - sparse_categorical_accuracy: 0.9440
Validation loss: 0.5167, Validation accuracy: 0.9440


In [25]:
# Get the model's predictions on the validation set
y_pred = np.argmax(bert_model.predict(val_dataset), axis=1)

# Print the classification report
from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred))

8/8 [==============================] - 103s 13s/step


AxisError: ignored

In [27]:
y_out = bert_model.predict(val_dataset)

8/8 [==============================] - 94s 11s/step


In [38]:
np.argmax(y_out[0], axis=1)

array([4, 1, 4, 2, 4, 4, 4, 4, 1, 4, 2, 3, 4, 0, 0, 1, 1, 4, 4, 4, 3, 4,
       4, 4, 1, 2, 1, 2, 4, 4, 1, 3, 1, 4, 4, 4, 1, 2, 2, 1, 2, 1, 1, 1,
       4, 1, 4, 4, 3, 2, 0, 1, 1, 4, 4, 1, 0, 1, 2, 4, 1, 1, 1, 1, 4, 1,
       4, 1, 1, 2, 1, 1, 4, 4, 1, 4, 3, 4, 4, 4, 1, 2, 4, 1, 4, 1, 4, 4,
       0, 2, 1, 4, 3, 4, 2, 3, 4, 4, 1, 4, 4, 1, 1, 4, 4, 1, 4, 1, 2, 2,
       1, 4, 2, 4, 4, 2, 4, 1, 4, 2, 2, 4, 4, 2, 4])

In [39]:
from sklearn.metrics import classification_report
print(classification_report(y_val, np.argmax(y_out[0], axis=1)))

              precision    recall  f1-score   support

           0       1.00      0.62      0.77         8
           1       0.97      1.00      0.99        38
           2       0.85      0.94      0.89        18
           3       1.00      0.70      0.82        10
           4       0.94      1.00      0.97        51

    accuracy                           0.94       125
   macro avg       0.95      0.85      0.89       125
weighted avg       0.95      0.94      0.94       125



In [32]:
len(y_val)

125

In [22]:
y_pred

0

In [23]:
val_dataset

<_BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 200), dtype=tf.int32, name=None)}, TensorSpec(shape=(None,), dtype=tf.int8, name=None))>